In [ ]:
import polars as pl

#1a. Pivot data
#Load the economics_long.csv dataset from the data/ folder. You'll notice that in the economics_long dataset's value column, there are different types of measurements. Transform this dataset so that the unique values for the variable field become columns of their own.

#Please write code below to pivot the data. The output should be a DataFrame. You must "pivot" the data to receive full credit on this question.

# Load economics_long data
economics_long = pl.read_csv("https://raw.githubusercontent.com/philhetzel/opan5510-class12/refs/heads/main/data/economics_long.csv")

# YOUR CODE HERE: Pivot the data so that each unique value in 'variable' becomes its own column
# Hint: Use the pivot() function
# The index should be 'date', columns should come from 'variable', and values from 'value'

economics_long = economics_long.with_columns(
    pl.col("date").str.strptime(pl.Date, "%Y-%m-%d")
)

# Pivot the data
economics_wide = economics_long.pivot(
    index="date",
    columns="variable",
    values="value",
    aggregate_function="first"  # Since each date-variable combo should be unique
)

# Display the pivoted dataframe
print(economics_wide)



shape: (574, 6)
┌────────────┬─────────┬────────────┬─────────┬─────────┬──────────┐
│ date       ┆ pce     ┆ pop        ┆ psavert ┆ uempmed ┆ unemploy │
│ ---        ┆ ---     ┆ ---        ┆ ---     ┆ ---     ┆ ---      │
│ date       ┆ f64     ┆ f64        ┆ f64     ┆ f64     ┆ f64      │
╞════════════╪═════════╪════════════╪═════════╪═════════╪══════════╡
│ 1967-07-01 ┆ 506.7   ┆ 198712.0   ┆ 12.6    ┆ 4.5     ┆ 2944.0   │
│ 1967-08-01 ┆ 509.8   ┆ 198911.0   ┆ 12.6    ┆ 4.7     ┆ 2945.0   │
│ 1967-09-01 ┆ 515.6   ┆ 199113.0   ┆ 11.9    ┆ 4.6     ┆ 2958.0   │
│ 1967-10-01 ┆ 512.2   ┆ 199311.0   ┆ 12.9    ┆ 4.9     ┆ 3143.0   │
│ 1967-11-01 ┆ 517.4   ┆ 199498.0   ┆ 12.8    ┆ 4.7     ┆ 3066.0   │
│ …          ┆ …       ┆ …          ┆ …       ┆ …       ┆ …        │
│ 2014-12-01 ┆ 12062.0 ┆ 319746.157 ┆ 7.6     ┆ 12.9    ┆ 8717.0   │
│ 2015-01-01 ┆ 12046.0 ┆ 319928.646 ┆ 7.7     ┆ 13.2    ┆ 8903.0   │
│ 2015-02-01 ┆ 12082.4 ┆ 320074.511 ┆ 7.9     ┆ 12.9    ┆ 8610.0   │
│ 2015-03-01 ┆ 121

/tmp/ipython-input-1914508044.py:20: DeprecationWarning: The argument `columns` for `DataFrame.pivot` is deprecated. It has been renamed to `on`.
  economics_wide = economics_long.pivot(


In [ ]:
import polars as pl
economics_long = pl.read_csv("https://raw.githubusercontent.com/philhetzel/opan5510-class12/refs/heads/main/data/economics_long.csv")
economics_long = economics_long.with_columns(
    pl.col("date").str.strptime(pl.Date, "%Y-%m-%d")
)

#1b. Calculate median unemployment for 2010-01-01 and beyond
#Next, compute the median unemployment metric (as defined by unemploy) for 2010-01-01 and beyond.

#Please write code below. The output should show the median value. You must use the dataframe created in 1a.

# YOUR CODE HERE: Filter for dates >= '2010-01-01' and calculate median unemployment
# Hint: You'll need to filter the dataframe and then calculate the median of the 'unemploy' column
filtered = economics_long.filter(
    (pl.col("date") >= pl.date(2010, 1, 1)) &
    (pl.col("variable") == "unemploy")
)

median_unemploy = filtered.select(pl.col("value").median()).item()

print(f"The median unemployment for 2010-01-01 and beyond is: {median_unemploy}")

The median unemployment for 2010-01-01 and beyond is: 12471.0


In [10]:
import polars as pl

#2a. Unpivot data
#The AirPassengers dataset is wide, which makes it difficult to aggregate. Transform the AirPassengers dataset from wide to long. The resulting dataset should have three columns: one column representing year, one column representing month, and one column representing the number of air passengers.

#Please write code below to unpivot the data. The output should be a DataFrame. You must "unpivot" the data to receive full credit on this question.

# Load AirPassengers data
# Note: You may need to specify separator if it's not comma-separated
air_passengers = pl.read_csv("https://raw.githubusercontent.com/philhetzel/opan5510-class12/refs/heads/main/data/AirPassengers.txt", separator="\t")  # Adjust separator if needed

# Display the original data structure
print("Original data shape:", air_passengers.shape)
print("\nFirst few rows:")
print(air_passengers.head())

# YOUR CODE HERE: Transform from wide to long format
# Hint: Use unpivot() function to unpivot the data
# You'll need to identify which columns represent months and unpivot them
# Use 'index' for columns to keep and 'on' for columns to unpivot

air_passengers_long = air_passengers.unpivot(
    index="Year",        # Keep the 'Year' column (corrected from 'year')
    on=[col for col in air_passengers.columns if col != "Year"], # Unpivot all other columns (the months)
    variable_name="month", # Name for the melted column headers
    value_name="passengers" # Name for the corresponding values
)

print(air_passengers_long)

Original data shape: (12, 13)

First few rows:
shape: (5, 13)
┌──────┬─────┬─────┬─────┬───┬─────┬─────┬─────┬─────┐
│ Year ┆ Jan ┆ Feb ┆ Mar ┆ … ┆ Sep ┆ Oct ┆ Nov ┆ Dec │
│ ---  ┆ --- ┆ --- ┆ --- ┆   ┆ --- ┆ --- ┆ --- ┆ --- │
│ i64  ┆ i64 ┆ i64 ┆ i64 ┆   ┆ i64 ┆ i64 ┆ i64 ┆ i64 │
╞══════╪═════╪═════╪═════╪═══╪═════╪═════╪═════╪═════╡
│ 1949 ┆ 112 ┆ 118 ┆ 132 ┆ … ┆ 136 ┆ 119 ┆ 104 ┆ 118 │
│ 1950 ┆ 115 ┆ 126 ┆ 141 ┆ … ┆ 158 ┆ 133 ┆ 114 ┆ 140 │
│ 1951 ┆ 145 ┆ 150 ┆ 178 ┆ … ┆ 184 ┆ 162 ┆ 146 ┆ 166 │
│ 1952 ┆ 171 ┆ 180 ┆ 193 ┆ … ┆ 209 ┆ 191 ┆ 172 ┆ 194 │
│ 1953 ┆ 196 ┆ 196 ┆ 236 ┆ … ┆ 237 ┆ 211 ┆ 180 ┆ 201 │
└──────┴─────┴─────┴─────┴───┴─────┴─────┴─────┴─────┘
shape: (144, 3)
┌──────┬───────┬────────────┐
│ Year ┆ month ┆ passengers │
│ ---  ┆ ---   ┆ ---        │
│ i64  ┆ str   ┆ i64        │
╞══════╪═══════╪════════════╡
│ 1949 ┆ Jan   ┆ 112        │
│ 1950 ┆ Jan   ┆ 115        │
│ 1951 ┆ Jan   ┆ 145        │
│ 1952 ┆ Jan   ┆ 171        │
│ 1953 ┆ Jan   ┆ 196        │
│ …    ┆ …     ┆ 

In [12]:
import polars as pl
air_passengers = pl.read_csv("https://raw.githubusercontent.com/philhetzel/opan5510-class12/refs/heads/main/data/AirPassengers.txt", separator="\t")  # Adjust separator if needed

#2b. Find the standard deviation between 1955 and 1960
#Next, compute the standard deviation of passengers between (and including) the years 1955 and 1960.

#Please write code below. The output should show the standard deviation. You must use the dataframe created in 2a.

# YOUR CODE HERE: Filter for years 1955-1960 and calculate standard deviation
# Hint: Filter the long format data for years between 1955 and 1960 (inclusive)
# Then calculate the standard deviation of the passenger values

air_passengers_long = air_passengers.unpivot(
    index="Year",
    on=[col for col in air_passengers.columns if col != "Year"],
    variable_name="month",
    value_name="passengers"
)

std_passengers = air_passengers_long.filter(
    pl.col("Year").is_between(1955, 1960)
).select(
    pl.col("passengers").std()
).item()

print(f"The standard deviation of airline passengers between 1955 and 1960 is: {std_passengers:.2f}")

The standard deviation of airline passengers between 1955 and 1960 is: 86.44


In [ ]:
import polars as pl

#3a. Pivot data
#Load the table2.csv dataset from the data/ folder. This contains data for country populations and tuberculosis case metrics. You'll notice that the metrics for cases and population are in the same column (count). Pivot the data so that the metrics for cases and population are in their own columns.

#Please write code below to pivot the data. The output should be a DataFrame. You must "pivot" the data to receive full credit on this question.

# Load table2 data
table2 = pl.read_csv("https://raw.githubusercontent.com/philhetzel/opan5510-class12/refs/heads/main/data/table2.csv")

# Display the original data structure
print("Original data:")
print(table2)

# YOUR CODE HERE: Pivot the data so 'cases' and 'population' are separate columns
# Hint: Use pivot() where the values in 'type' column become new column names
# and values come from the 'count' column
table2_wide = table2.pivot(
    index=["country", "year"],
    columns="type",
    values="count"
)

print(table2_wide)


Original data:
shape: (12, 5)
┌──────────┬─────────────┬──────┬────────────┬────────────┐
│ rownames ┆ country     ┆ year ┆ type       ┆ count      │
│ ---      ┆ ---         ┆ ---  ┆ ---        ┆ ---        │
│ i64      ┆ str         ┆ i64  ┆ str        ┆ i64        │
╞══════════╪═════════════╪══════╪════════════╪════════════╡
│ 1        ┆ Afghanistan ┆ 1999 ┆ cases      ┆ 745        │
│ 2        ┆ Afghanistan ┆ 1999 ┆ population ┆ 19987071   │
│ 3        ┆ Afghanistan ┆ 2000 ┆ cases      ┆ 2666       │
│ 4        ┆ Afghanistan ┆ 2000 ┆ population ┆ 20595360   │
│ 5        ┆ Brazil      ┆ 1999 ┆ cases      ┆ 37737      │
│ …        ┆ …           ┆ …    ┆ …          ┆ …          │
│ 8        ┆ Brazil      ┆ 2000 ┆ population ┆ 174504898  │
│ 9        ┆ China       ┆ 1999 ┆ cases      ┆ 212258     │
│ 10       ┆ China       ┆ 1999 ┆ population ┆ 1272915272 │
│ 11       ┆ China       ┆ 2000 ┆ cases      ┆ 213766     │
│ 12       ┆ China       ┆ 2000 ┆ population ┆ 1280428583 │
└─────────

/tmp/ipython-input-2591856578.py:18: DeprecationWarning: The argument `columns` for `DataFrame.pivot` is deprecated. It has been renamed to `on`.
  table2_wide = table2.pivot(


In [ ]:
import polars as pl
table2 = pl.read_csv("https://raw.githubusercontent.com/philhetzel/opan5510-class12/refs/heads/main/data/table2.csv")

#3b. Calculate highest cases/population ratio for 1999
#Now that you have cases and population in their own columns, you can perform analysis. Create a new column in this dataset called ratio that divides cases by population. Next, filter the data to only include rows from the year of 1999. What is the country with the highest cases/population ratio?

#Please write code below. You must use the dataframe created in 3a.

# YOUR CODE HERE:
# 1. Add a 'ratio' column that calculates cases/population
# 2. Filter for year 1999
# 3. Find the country with the highest ratio

table2_wide = table2.pivot(
    index=["country", "year"],
    columns="type",
    values="count"
)

table2_with_ratio = table2_wide.with_columns(
    (pl.col("cases") / pl.col("population")).alias("ratio")
)

year_1999 = table2_with_ratio.filter(pl.col("year") == 1999)

highest_ratio_country = year_1999.sort("ratio", descending=True).select("country").row(0)[0]

print("Data for 1999 with ratios:")
print(year_1999)

print(f"\nThe country with the highest cases/population ratio in 1999 is: {highest_ratio_country}")

Data for 1999 with ratios:
shape: (3, 5)
┌─────────────┬──────┬────────┬────────────┬──────────┐
│ country     ┆ year ┆ cases  ┆ population ┆ ratio    │
│ ---         ┆ ---  ┆ ---    ┆ ---        ┆ ---      │
│ str         ┆ i64  ┆ i64    ┆ i64        ┆ f64      │
╞═════════════╪══════╪════════╪════════════╪══════════╡
│ Afghanistan ┆ 1999 ┆ 745    ┆ 19987071   ┆ 0.000037 │
│ Brazil      ┆ 1999 ┆ 37737  ┆ 172006362  ┆ 0.000219 │
│ China       ┆ 1999 ┆ 212258 ┆ 1272915272 ┆ 0.000167 │
└─────────────┴──────┴────────┴────────────┴──────────┘

The country with the highest cases/population ratio in 1999 is: Brazil


/tmp/ipython-input-1879828074.py:14: DeprecationWarning: The argument `columns` for `DataFrame.pivot` is deprecated. It has been renamed to `on`.
  table2_wide = table2.pivot(
